In [1]:
# Import the package
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from transformers import BertTokenizer, BertModel, AdamW
import numpy as np
import pandas as pd
from tqdm import tqdm
import xml.etree.ElementTree as ET
import nltk
from nltk.corpus import wordnet as wn
from collections import namedtuple

In [6]:
# Settings for loading training data
semcor_training_xml_path = '../data/WSD_Evaluation_Framework/Training_Corpora/Semcor/semcor.data.xml'
semcor_training_gk_path = '../data/WSD_Evaluation_Framework/Training_Corpora/Semcor/semcor.gold.key.txt'

In [7]:
def load_xml_data(xml_file_path=''):
    tree = ET.parse(xml_file_path)
    root = tree.getroot()
    data = []
    for text in root.findall('text'):
        for sentence in text.findall('sentence'):
            sentence_id = sentence.get('id')
            sentence_text = ' '.join([element.text for element in sentence])
            for instance in sentence.findall('instance'):
                instance_id = instance.get('id')
                lemma = instance.get('lemma')
                pos = instance.get('pos')
                word = instance.text
                data.append([sentence_id, instance_id, lemma, pos, word, sentence_text])
    columns = ['sentence_id', 'instance_id', 'lemma', 'pos', 'word', 'sentence_text']
    xml_data = pd.DataFrame(data, columns=columns)
    return xml_data


# Load xml training data from semcor
semcor_training_xml = load_xml_data(semcor_training_xml_path)
display(semcor_training_xml.head())

,sentence_id,instance_id,lemma,pos,word,sentence_text
0,d000.s000,d000.s000.t000,long,ADJ,long,How long has it been since you reviewed the ob...
1,d000.s000,d000.s000.t001,be,VERB,been,How long has it been since you reviewed the ob...
2,d000.s000,d000.s000.t002,review,VERB,reviewed,How long has it been since you reviewed the ob...
3,d000.s000,d000.s000.t003,objective,NOUN,objectives,How long has it been since you reviewed the ob...
4,d000.s000,d000.s000.t004,benefit,NOUN,benefit,How long has it been since you reviewed the ob...


In [8]:
def load_gold_keys(gold_key_file_path=''):
    gold_key_data = []
    with open(gold_key_file_path, 'r') as file:
        for line in file:
            parts = line.strip().split()
            instance_id = parts[0]
            sense_id = parts[1]
            gold_key_data.append([instance_id, sense_id])

    # Create a DataFrame
    gold_key_columns = ['instance_id', 'sense_id']
    gold_key_df = pd.DataFrame(gold_key_data, columns=gold_key_columns)
    return gold_key_df

# Load gold key training data from semcor
semcor_training_gk = load_gold_keys(semcor_training_gk_path)
display(semcor_training_gk.head())

,instance_id,sense_id
0,d000.s000.t000,long%3:00:02::
1,d000.s000.t001,be%2:42:03::
2,d000.s000.t002,review%2:31:00::
3,d000.s000.t003,objective%1:09:00::
4,d000.s000.t004,benefit%1:21:00::


In [9]:
# Merge data from two file
semcor_training_merged = pd.merge(semcor_training_xml, semcor_training_gk, on='instance_id', how='inner')
display(semcor_training_merged.head())

,sentence_id,instance_id,lemma,pos,word,sentence_text,sense_id
0,d000.s000,d000.s000.t000,long,ADJ,long,How long has it been since you reviewed the ob...,long%3:00:02::
1,d000.s000,d000.s000.t001,be,VERB,been,How long has it been since you reviewed the ob...,be%2:42:03::
2,d000.s000,d000.s000.t002,review,VERB,reviewed,How long has it been since you reviewed the ob...,review%2:31:00::
3,d000.s000,d000.s000.t003,objective,NOUN,objectives,How long has it been since you reviewed the ob...,objective%1:09:00::
4,d000.s000,d000.s000.t004,benefit,NOUN,benefit,How long has it been since you reviewed the ob...,benefit%1:21:00::


In [10]:
sense_id_cache = {}
def format_sense_id(sense_id):
    if sense_id not in sense_id_cache:
        sense_id_cache[sense_id] = wn.lemma_from_key(sense_id).synset().name()
    return sense_id_cache[sense_id]
    
semcor_training_merged['formatted_sense_id'] = semcor_training_merged['sense_id'].apply(format_sense_id)

# We keep those columns for now
keys_to_keep = ['lemma', 'word', 'sentence_text', 'formatted_sense_id']
semcor_training_merged = semcor_training_merged[keys_to_keep]

display(semcor_training_merged.head())

,lemma,word,sentence_text,formatted_sense_id
0,long,long,How long has it been since you reviewed the ob...,long.a.01
1,be,been,How long has it been since you reviewed the ob...,be.v.01
2,review,reviewed,How long has it been since you reviewed the ob...,review.v.01
3,objective,objectives,How long has it been since you reviewed the ob...,aim.n.02
4,benefit,benefit,How long has it been since you reviewed the ob...,benefit.n.01


In [12]:
###### Setting to load nball embeddings
nball_small_path = '../data/sample_mammal/nball.txt'

def load_ball_embeddings(bFile):
    Ball = namedtuple('Ball', ['center', 'distance', 'radius'])
    print("loading balls....")
    bdic = dict()
    with open(bFile, 'r') as w2v:
        for line in w2v.readlines():
            wlst = line.strip().split()
            center = list(map(float, wlst[1:-2]))
            distance = float(wlst[-2])
            radius = float(wlst[-1])
            # Calculate the real center coordinates
            real_center = [distance * coord for coord in center]
            # Store the information in the dictionary using the named tuple
            bdic[wlst[0]] = Ball(center=real_center, distance=distance, radius=radius)
    print(len(bdic), 'balls are loaded\n')
    return bdic


# Load ball embeddings
nball = load_ball_embeddings(nball_small_path)

# Check the length of the nball embeddings
nball_len =[]
for i, (key, value) in enumerate(nball.items()):
    value_len = len(nball[key].center)
    if value_len not in nball_len:
        nball_len.append(value_len)
print(f'The length of nball embeddings: {nball_len}')

print(nball['mammal.n.01'].center)
print(nball['mammal.n.01'].distance)
print(nball['mammal.n.01'].radius)

loading balls....
388 balls are loaded

The length of nball embeddings: [160]
[9285.874696624029, -2831.0731960533985, -5411.5754703092125, 3155.586178547479, 9616.250093137523, 8773.95357563832, -6489.03043403883, -11213.285138104024, 18446.747597543756, -6378.590118318234, 6159.033257784651, 455.2731816464883, 17035.986030290933, 9392.627364816282, 6494.703737928586, 1341.9254800571066, 4968.1122162599, 168.8091572397092, -11585.832093531379, 7148.268346028553, -11089.41800317767, -5555.204613784883, -1312.9916302193483, -3979.91723372912, 5997.344096926587, 2099.406104404449, -1954.9259653453128, -6383.790646883844, -2217.127160116899, -870.3793717534998, 5279.292934613052, -8745.870721384026, 4289.01774065603, -2987.3727182161947, -796.3049339662437, 1759.8588666025137, 2669.9513655843093, -3057.8162415140064, 2300.524727296322, -223.3296076202706, -13191.377094332509, -8040.773602952104, 8621.814476328012, 12485.996310706098, 5211.686063260118, 3003.257969107513, 1774.325791521393

In [13]:
# For now, we train on exist nball

print(f'Original length of the training set:{len(semcor_training_merged)}')
semcor_training_origian = semcor_training_merged[semcor_training_merged['formatted_sense_id'].isin(nball.keys())].copy()
print(f'Actual data used of the training set:{len(semcor_training_origian)}')

semcor_training = semcor_training_origian
# semcor_training = pd.concat([semcor_training_origian] * 10, ignore_index=True)
print(f'Actual data used after copy:{len(semcor_training)}')
display(semcor_training.head())

Original length of the training set:226036
Actual data used of the training set:352
Actual data used after copy:352


,lemma,word,sentence_text,formatted_sense_id
3727,man,men,Whereas the eighteenth century had been a time...,homo.n.02
6098,horse,horse,One wrote : `` [ I am so hungry ] I could eat ...,horse.n.01
6151,wildcat,wildcat,"She is well-educated and refined , all wildcat...",wildcat.n.03
7494,lion,Lions,"To find out , we traveled throughout that part...",lion.n.01
7929,elephant,elephants,The Prince visited the hospital of Operation B...,elephant.n.01


In [14]:
# Setting for the model choice
models = {
    "BERT-Base": "bert-base-uncased", #768, 12L, 12A
    "BERT-Large": "bert-large-uncased", #1024, 24L, 16A
    "BERT-Medium": "google/bert_uncased_L-8_H-512_A-8", #512, 8L, 8A
    "BERT-Small": "google/bert_uncased_L-4_H-256_A-4", #256, 4L, 4A
    "BERT-Mini": "google/bert_uncased_L-4_H-128_A-2",#128, 4L, 2A
    "BERT-Tiny": "google/bert_uncased_L-2_H-128_A-2"#128, 2L, 2A
}

# With our nball dimention 162, we choose bert small with 256 dimentions
model_name = models["BERT-Small"]
target_dim = 256  # Dimension of BERT-Small

In [15]:
# Get sense index
sense_labels = list(nball.keys())
sense_index = {sense: idx for idx, sense in enumerate(sense_labels)}
semcor_training.loc[:,'sense_idx'] = semcor_training['formatted_sense_id'].map(sense_index)

original_dim = len(nball[sense_labels[0]].center)
padding_size = target_dim - original_dim  # Adjust padding size based on the new dimension

# Process each embedding
padded_embeddings = []
nball_radius = []
scalling_factor = 1
for label in sense_labels:
    # # Pad the trimmed embedding to match the target dimension
    # padded_embedding = np.pad(nball[label].center, (0, padding_size), 'constant', constant_values=0)
    # # Add to the list of padded embeddings
    # padded_embeddings.append(scalling_factor*padded_embedding)
    padded_embeddings.append(nball[label].center)
    nball_radius.append(nball[label].radius)

sense_embeddings = torch.tensor(np.array(padded_embeddings), dtype=torch.float64)
nball_radius = torch.tensor(np.array(nball_radius), dtype=torch.float64)

print(f'Total data of nball embeddings:{len(sense_embeddings)}')
print(f'The length after padding: {len(sense_embeddings[0])}')

Total data of nball embeddings:388
The length after padding: 160


In [12]:
# Tokenize the sentence
# Initialize BERT tokenizer
tokenizer = BertTokenizer.from_pretrained(model_name)

# Could be problem here, as we always fid
def find_word_index(sentence_ids, word):
    word_tokens = tokenizer.tokenize(word)
    word_ids = tokenizer.convert_tokens_to_ids(word_tokens)
    for i in range(len(sentence_ids) - len(word_tokens) + 1):
        if sentence_ids[i:i+len(word_tokens)].tolist() == word_ids:
            return i
    return -1

def tokenize_data(df):
    # Tokenize all sentences
    print("Tokenizing sentences...")
    tokenized_data = tokenizer(list(df['sentence_text']), padding=True, truncation=True, return_tensors="pt", max_length=512)
    input_ids = tokenized_data['input_ids']
    attention_masks = tokenized_data['attention_mask']

    # Progress bar for calculating word indices
    print("Calculating word indices...")
    # pbar = tqdm(total=df.shape[0], desc="Calculating word indices")
    word_indices = []
    for sentence_ids, word in zip(input_ids, df['word']):
        word_indices.append(find_word_index(sentence_ids, word))
        # pbar.update(1)  # Update progress for each word index found

    # print(f"Length input_ids:{len(input_ids)}\n Length attention_mask:{len(attention_masks)}\n Length \
    # word_index:{len(word_indices)}\n Length dataframe:{len(df)}")
    df.loc[:, 'input_ids'] = input_ids.tolist()
    df.loc[:, 'attention_mask'] = attention_masks.tolist()
    df.loc[:, 'word_index'] = word_indices

    print('Tokenizing finished!')
    # pbar.close()  # Close the progress bar after completion


tokenize_data(semcor_training)
display(semcor_training.head())

Tokenizing sentences...
Calculating word indices...
Tokenizing finished!


,lemma,word,sentence_text,formatted_sense_id,sense_idx,input_ids,attention_mask,word_index
3727,man,men,Whereas the eighteenth century had been a time...,homo.n.02,132,"[101, 6168, 1996, 12965, 2301, 2018, 2042, 103...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",24
6098,horse,horse,One wrote : `` [ I am so hungry ] I could eat ...,horse.n.01,261,"[101, 2028, 2626, 1024, 1036, 1036, 1031, 1045...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",19
6151,wildcat,wildcat,"She is well-educated and refined , all wildcat...",wildcat.n.03,54,"[101, 2016, 2003, 2092, 1011, 5161, 1998, 1551...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",10
7494,lion,Lions,"To find out , we traveled throughout that part...",lion.n.01,62,"[101, 2000, 2424, 2041, 1010, 2057, 6158, 2802...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",50
7929,elephant,elephants,The Prince visited the hospital of Operation B...,elephant.n.01,104,"[101, 1996, 3159, 4716, 1996, 2902, 1997, 3169...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",27


In [13]:
# Setting, small batch size here so that do not out of cuda memory
batch_size = 32

# Create tensor dataset to speed learning
# Convert lists of lists into tensors
all_input_ids = torch.stack([torch.tensor(ids, dtype=torch.long) for ids in semcor_training['input_ids']])
all_attention_masks = torch.stack([torch.tensor(mask, dtype=torch.long) for mask in semcor_training['attention_mask']])
all_word_indices = torch.tensor(semcor_training['word_index'].tolist(), dtype=torch.long)
all_senses = torch.tensor(semcor_training['sense_idx'].tolist(), dtype=torch.long)

# Create a TensorDataset
dataset = TensorDataset(all_input_ids, all_attention_masks, all_word_indices, all_senses)

# Use DataLoader to handle batching
dataloader = DataLoader(dataset, batch_size, shuffle=True)

In [18]:
# Training the model
# Loss
from torch.optim.lr_scheduler import StepLR

# Set the device to CUDA if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# top_values, top_indices = torch.topk(nball_radius, 10)
# top_indices = top_indices.to(device)

def ball_inclution_loss_soft_margin(word_embeddings, centers, radius):
    distances = torch.norm(word_embeddings - centers, dim=1)
    soft_margin = torch.clamp(distances - radius, min=0)
    loss = torch.mean(soft_margin ** 2)
    return loss, distances
 

class CustomBertModel(nn.Module):
    def __init__(self, bert_model_name):
        super(CustomBertModel, self).__init__()
        self.bert = BertModel.from_pretrained(bert_model_name)
        self.projection = nn.Linear(self.bert.config.hidden_size, 160)
        
    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        hidden_states = outputs.last_hidden_state
        projected_hidden_states = self.projection(hidden_states)
        return projected_hidden_states


def ball_inclusion_loss(word_embeddings, centers, radius, labels):
    distances = torch.norm(word_embeddings - centers, dim=1)
    # Compute a binary mask for embeddings within the ball radius
    within_ball = (distances <= radius).float()
    loss = F.binary_cross_entropy(within_ball, labels.float(), reduction='mean')
    return loss

def ball_inclusion_loss_cosin(word_embeddings, centers, radius, labels):
    norm_center = torch.norm(centers, p=2, dim=1)
    norm_word = torch.norm(word_embeddings, p=2, dim=1)
    xc = F.cosine_similarity(centers, word_embeddings)
    distances = norm_center**2 + norm_word**2 - 2 * xc * norm_center * norm_word
    # Compute a binary mask for embeddings within the ball radius
    within_ball = (distances <= radius**2).float()
    loss = F.binary_cross_entropy(within_ball, labels.float(), reduction='mean')
    return loss

def ball_inclusion_loss_old(word_embeddings, centers, radius, labels):
    distances = torch.norm(word_embeddings - centers, dim=1)
    # Compute the loss for each word embedding
    losses = F.relu(distances - radius)
    # Aggregate the losses into a single scalar value
    loss = losses.mean()
    return loss

def ball_distances(word_embeddings, centers, radius, labels):
    distances = torch.norm(word_embeddings - centers, dim=1)
    loss = distances.mean()
    return loss

def ball_inclution_loss_soft_margin(word_embeddings, centers, radius):
    distances = torch.norm(word_embeddings - centers, dim=1)
    soft_margin = torch.clamp(distances - radius, min=0)
    loss = torch.mean(soft_margin ** 2)
    return loss, distances
    
# Model setting
# model = BertModel.from_pretrained(model_name).to(device)
model = CustomBertModel(model_name).to(device)
model.train() 
sense_embeddings = sense_embeddings.to(device)  # Move sense embeddings to GPU
loss_fn = ball_inclution_loss_soft_margin
optimizer = optim.Adam([
    {'params': model.parameters()}
    # {'params': sense_embeddings},
    # {'params': nball_radius}
], lr=0.1)

scheduler = StepLR(optimizer, step_size=50, gamma=0.1)


# Training loop
num_epochs = 200
last_loss = 0
for epoch in range(num_epochs):
    total_loss = 0
    total_dis = 0
    total_radius = 0
    progress_bar = tqdm(dataloader, desc=f'Epoch {epoch+1}/{num_epochs}', leave=True, position=0)
    for batch in progress_bar:
        # Send batch data to the device (GPU)
        batch_input_ids, batch_attention_masks, batch_word_indices, batch_sense_indices = [b.to(device) for b in batch]

        optimizer.zero_grad()
        
        # Forward pass
        hidden_states = model(input_ids=batch_input_ids, attention_mask=batch_attention_masks)
        # hidden_states = outputs.last_hidden_state
        
        # Retrieve embeddings for specific word indices
        word_embeddings = torch.stack([hidden_states[i, idx, :] for i, idx in enumerate(batch_word_indices)])
        
        # Retrieve the corresponding sense embeddings
        center = sense_embeddings[batch_sense_indices].to(dtype=torch.float32).to(device)
        radius = nball_radius[batch_sense_indices].to(device)

        # Labels tensor indicating that embeddings should be inside the ball
        labels = torch.ones(word_embeddings.size(0), device=device, requires_grad=True)
        # Calculate loss
            
        loss, dis = loss_fn(word_embeddings, center, radius)
        
        # Backward pass
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        total_dis += dis.sum().item()
        total_radius += radius.sum().item()
        progress_bar.set_postfix(loss=loss.item())

    
    print(f'Epoch {epoch + 1}, Loss: {total_loss / len(dataloader)}, improve:{(last_loss - total_loss) / len(dataloader)}')
    print(f'distance:{total_dis / len(dataloader)}, radius:{total_radius / len(dataloader)}, differences:{(total_dis-total_radius) / len(dataloader)}')
    last_loss = total_loss

Using device: cuda


Epoch 1/200: 100%|██████████████████████████████████████████████████████| 11/11 [00:01<00:00, 10.64it/s, loss=4.88e+11]


Epoch 1, Loss: 744610272905.0912, improve:-744610272905.0912
distance:24525259.09090909, radius:6705831.376555665, differences:17819427.714353427


Epoch 2/200: 100%|██████████████████████████████████████████████████████| 11/11 [00:00<00:00, 14.96it/s, loss=9.03e+11]


Epoch 2, Loss: 733067612255.8965, improve:11542660649.19469
distance:24229425.90909091, radius:6705831.376555665, differences:17523594.532535244


...

Epoch 198, Loss: 410391599635.3901, improve:362400333.8552912
distance:20858576.636363637, radius:6705831.376555665, differences:14152745.259807972


Epoch 199/200: 100%|████████████████████████████████████████████████████| 11/11 [00:00<00:00, 15.41it/s, loss=3.63e+11]


Epoch 199, Loss: 410693876824.8022, improve:-302277189.4121094
distance:21028292.545454547, radius:6705831.3765556635, differences:14322461.16889888


Epoch 200/200: 100%|████████████████████████████████████████████████████| 11/11 [00:00<00:00, 15.42it/s, loss=4.61e+11]

Epoch 200, Loss: 410310226358.09515, improve:383650466.70703125
distance:20927055.636363637, radius:6705831.376555666, differences:14221224.259807972


In [24]:
# Assuming that 'dataloader' for evaluation is available as eval_dataloader

# Switch model to evaluation mode
eval_dataloader = dataloader
model.eval()

# Store evaluation results
eval_loss = 0
accuracy = 0  # Placeholder for accuracy calculation

# Disable gradient computation for evaluation to save memory and computations
with torch.no_grad():
    for batch in tqdm(eval_dataloader, desc="Evaluation", leave=True, position=0):
        # Send batch data to the device (GPU)
        batch_input_ids, batch_attention_masks, batch_word_indices, batch_sense_indices = [b.to(device) for b in batch]

        # Forward pass
        hidden_states = model(input_ids=batch_input_ids, attention_mask=batch_attention_masks)
        # hidden_states = outputs.last_hidden_state

        # # Retrieve embeddings for specific word indices
        word_embeddings = torch.stack([hidden_states[i, idx, :] for i, idx in enumerate(batch_word_indices)])
        # src = batch_input_ids * math.sqrt(d_model)  # Scale embedding according to the model size
        # src = positional_encoding(src)  # Add positional encoding
        # output = model(src)

        # Retrieve the corresponding sense embeddings and radius
        center = sense_embeddings[batch_sense_indices].to(dtype=torch.float32).to(device)
        radius = nball_radius[batch_sense_indices].to(device)

        # Labels tensor (usually true for all points in eval since you are evaluating the inclusion)
        labels = torch.ones(word_embeddings.size(0), device=device)

        # Calculate loss
        loss = ball_inclusion_loss(word_embeddings, center, radius, labels)
        eval_loss += loss.item()

        # Optionally calculate accuracy or other metrics
        # For instance, if accuracy is based on correct inclusions
        predictions = (torch.norm(1e4 * word_embeddings - center, dim=1) <= radius).float()
        accuracy += (predictions == labels).float().mean().item()

# Compute average loss and accuracy
eval_loss /= len(eval_dataloader)
accuracy /= len(eval_dataloader)

print(f"Average Evaluation Loss: {eval_loss}")
print(f"Accuracy: {accuracy * 100:.2f}%")

Evaluation: 100%|██████████████████████████████████████████████████████████████████████| 11/11 [00:00<00:00, 27.73it/s]


Average Evaluation Loss: 74.43181818181819
Accuracy: 0.00%


In [ ]:
# Training the model
# Set the device to CUDA if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# Model setting
model = BertModel.from_pretrained(model_name).to(device)
model.train() 
sense_embeddings = sense_embeddings.to(device)  # Move sense embeddings to GPU
loss_fn = nn.CosineEmbeddingLoss()
optimizer = optim.Adam(model.parameters(), lr=2e-5)


# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    total_loss = 0
    progress_bar = tqdm(dataloader, desc=f'Epoch {epoch+1}/{num_epochs}', leave=True, position=0)
    for batch in progress_bar:
        # Send batch data to the device (GPU)
        batch_input_ids, batch_attention_masks, batch_word_indices, batch_sense_indices = [b.to(device) for b in batch]

        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(input_ids=batch_input_ids, attention_mask=batch_attention_masks)
        hidden_states = outputs.last_hidden_state
        
        # Retrieve embeddings for specific word indices
        word_embeddings = torch.stack([hidden_states[i, idx, :] for i, idx in enumerate(batch_word_indices)])
        
        # Retrieve the corresponding sense embeddings
        target_embeddings = sense_embeddings[batch_sense_indices]

        # Labels tensor indicating that embeddings should be similar
        labels = torch.ones(word_embeddings.size(0), device=device)

        # Calculate loss
        loss = loss_fn(word_embeddings, target_embeddings, labels)
        
        # Backward pass
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        progress_bar.set_postfix(loss=loss.item())

    print(f'Epoch {epoch + 1}, Loss: {total_loss / len(dataloader)}')

In [ ]:
columns_to_display = ['formatted_sense_id', 'predicted_sense_id', 'pred_distance', 'target_distance']
display(semcor_training[columns_to_display].head())
correct_predictions = semcor_training['formatted_sense_id'] == semcor_training['predicted_sense_id']
accuracy = correct_predictions.mean()

print(f"Model Accuracy: {accuracy * 100:.2f}%")

In [21]:
nball_radius.sort()

torch.return_types.sort(
values=tensor([1.0000e-04, 1.0000e-04, 1.0000e-04, 1.0000e-04, 1.0010e-04, 1.0509e-04,
        1.0509e-04, 1.0509e-04, 1.0509e-04, 1.0509e-04, 1.0509e-04, 1.0828e-04,
        1.0828e-04, 1.0828e-04, 1.1779e-04, 1.1779e-04, 1.1779e-04, 1.1779e-04,
        1.1779e-04, 1.1779e-04, 1.1779e-04, 1.1779e-04, 1.1779e-04, 1.1779e-04,
        1.1779e-04, 1.1779e-04, 1.1779e-04, 1.1779e-04, 1.1779e-04, 1.1779e-04,
        1.2026e-04, 1.2538e-04, 1.3136e-04, 1.3826e-04, 1.4571e-04, 1.4571e-04,
        1.4571e-04, 1.4571e-04, 1.5365e-04, 1.5365e-04, 1.6231e-04, 1.6231e-04,
        1.6231e-04, 1.7599e-04, 1.9554e-04, 1.9554e-04, 1.9554e-04, 1.9554e-04,
        2.0536e-04, 2.0536e-04, 2.3295e-04, 2.3295e-04, 2.3295e-04, 2.3295e-04,
        2.3295e-04, 2.3295e-04, 2.3295e-04, 2.3295e-04, 2.3295e-04, 2.3295e-04,
        2.3295e-04, 2.3295e-04, 2.3295e-04, 2.3295e-04, 2.3295e-04, 2.3295e-04,
        2.3295e-04, 2.3295e-04, 2.3295e-04, 2.3795e-04, 2.4823e-04, 2.5932e-04,
        